In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import pyarrow
import fastparquet
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import numpy as np
import joblib
pd.set_option('display.max_colwidth', None)

In [2]:
recipes = pd.read_parquet('../dataset/recipes.parquet.gzip')

In [12]:
recipes.vote_count.min()

15

# tf idf

In [3]:
df = pd.read_parquet('../dataset/recipes.parquet.gzip')
df = df[df['description'].notna()]
df.reset_index(inplace=True)

In [4]:
def process_text(text):
    # replace multiple spaces with one
    text = ' '.join(text.split())
    # lowercase
    text = text.lower()
 
    return text

df['description'] = df.apply(lambda x: process_text(x.description),axis=1)

In [5]:
tf_idf = TfidfVectorizer(stop_words='english')
 
tf_idf_matrix = tf_idf.fit_transform(df['description']);

In [6]:
cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

In [7]:
def index_from_title(df,title):
    return df[df['recipe_title']==title].index.values[0]


# function that returns the title of the movie from its index
def title_from_index(df,index):
    return df[df.index==index].recipe_title.values[0]


# generating recommendations for given title
def recommendations(recipe_title, df,cosine_similarity_matrix,number_of_recommendations):
    index = index_from_title(df,recipe_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(number_of_recommendations+1)]]

    return df['recipe_title'].iloc[recommendations_indices]

In [8]:
dfkaki = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')

recommendations('Goan Sorak Curry Recipe', dfkaki, cosine_similarity_matrix, 13)

42                       Goan Kaju Curry Recipe (Spicy Goan Cashew Nut Curry)
2197    कोल्हापुरी पालक अंडे की करी रेसिपी - Kolhapuri Palak Egg Curry Recipe
1586                                                    Macaroni Chaat Recipe
72                                          Vanilla Dutch Baby Pancake Recipe
253                Chettinad Chicken Roast Recipe | Chicken Ghee Roast Recipe
2551    Cauliflower Au Gratin Recipe - French Style Cauliflower & Cheese Bake
6465                          Uttarakhand Mooli Thechua Recipe (Radish Salad)
4356                                     Eggless Checker Board Cookies Recipe
150                   Mangalorean Manoli Curry Recipe - Indian Gherkins Curry
742                                  Parsi Salli Murgh Recipe - Salli Marghi 
2253                    Rajasthani Style Sukhe Gatte Capsicum Ki Sabzi Recipe
55                               Dhaba Style Dal Fry Recipe | Mixed Dal Tadka
3407                                                     Tomato 

In [ ]:
df

# vectorize

In [35]:
recipes = pd.read_parquet('../dataset/recipes.parquet.gzip')

In [36]:
df = recipes

In [38]:
def return_values(value):
    values = []
    if value is not None:
        values.append(value.lower().replace(" ",""))

    return ' '.join(values)

def return_list_values(value):
    values = []
    if value is not None:
        for item in value:
            values.append(item.lower().replace(" ",""))

    return ' '.join(values)

df['record_health']=df.apply(lambda x: return_values(x.record_health),axis=1)
df['cuisine']=df.apply(lambda x: return_values(x.cuisine),axis=1)
df['course']=df.apply(lambda x: return_values(x.course),axis=1)
df['diet']=df.apply(lambda x: return_values(x.diet),axis=1)
df['category']=df.apply(lambda x: return_values(x.category),axis=1)
df['tags']=df.apply(lambda x: return_list_values(x.tags),axis=1)


In [43]:
w_record_health = 2
w_course = 1
w_cuisine = 5
w_diet = 3
w_prep_time = 6
w_cook_time = 5
w_category = 1
w_tags = 3
# function for merging features
 
def concatenate_features(df_row):
    return ' '.join([df_row['record_health']]*w_record_health) +' ' + \
            ' '.join([df_row['cuisine']]*w_cuisine)+' '+ \
            ' '.join([df_row['course']]*w_course)+' '+ \
            ' '.join([df_row['diet']]*w_diet)+' '+ \
            ' '.join(str([df_row['prep_time']])*w_prep_time)+' '+ \
            ' '.join(str([df_row['cook_time']])*w_cook_time)+' '+ \
            ' '.join([df_row['category']]*w_category)+' '+ \
            ' '.join([df_row['tags']]*w_tags)


In [44]:
df['features'] = df.apply(concatenate_features,axis=1)

In [55]:
# pre-processing text of features

def process_text(text):
    # replace multiple spaces with one
    text = ' '.join(text.split())
    # lowercase
    text=text.lower()

    return text

df['features'] = df.apply(lambda x: process_text(x.features),axis=1)


In [56]:
vect = CountVectorizer(stop_words='english')

vect_matrix = vect.fit_transform(df['features'])

cosine_similarity_matrix_count_based = cosine_similarity(vect_matrix, vect_matrix)

In [57]:
recommendations('Goan Sorak Curry Recipe', dfkaki, cosine_similarity_matrix_count_based, 13)

2500                           Goan Batata Recheado Recipe With Carrots & Beans
636                                                 Goan Prawns Vindaloo Recipe
2479                                                    Goan Gawar Bhaji Recipe
151                                                 Goan Vegetable Curry Recipe
6743           गोअन एग ड्राप करी रेसिपी - Goan Egg Drop Curry (Recipe In Hindi)
458                    Mushroom Caldine Recipe | Vegetarian Goan Mushroom Curry
5576            Doodanche Fov Recipe (Goan Style Milk Beaten Rice/Poha In Milk)
6537                     दही वाली भिन्डी रेसिपी - Dahi Bhindi (Recipe In Hindi)
6235                                                       Prawn Balchao Recipe
6279    The Goan Kelyachyo Fodi Recipe (Spicy & Crispy Pan Fried Banana Recipe)
5376                         Goan Style Dhayanche Fov Recipe (Curd Poha Recipe)
6653     Goan Style Sando Recipe (Steamed Rice Cakes Filled With Sweet Coconut)
7943                                    

# load model

In [13]:
def _process_text(text):
    text = ' '.join(text.split())
    text = text.lower()

    return text


def _calc_model(all_recipes):
    df = all_recipes[all_recipes['description'].notna()]
    df['description'] = df.apply(lambda x: _process_text(x.description), axis=1)

    tf_idf = TfidfVectorizer(stop_words='english')
    tf_idf_matrix = tf_idf.fit_transform(df['description'])
    cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

    joblib.dump(cosine_similarity_matrix, '../models/tf_idf.joblib')


In [14]:
all_recipes = pd.read_parquet('../dataset/recipes.parquet.gzip')

In [15]:
_calc_model(all_recipes)

In [18]:
all_recipes.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008],
           dtype='int64', length=7685)

In [13]:
cosine_similarity_matrix

array([[1.        , 0.0337129 , 0.05395738, ..., 0.01681173, 0.01764909,
        0.01355815],
       [0.0337129 , 1.        , 0.02647728, ..., 0.02879546, 0.00495894,
        0.03535348],
       [0.05395738, 0.02647728, 1.        , ..., 0.03452351, 0.01621537,
        0.0127036 ],
       ...,
       [0.01681173, 0.02879546, 0.03452351, ..., 1.        , 0.00800622,
        0.06164382],
       [0.01764909, 0.00495894, 0.01621537, ..., 0.00800622, 1.        ,
        0.00763376],
       [0.01355815, 0.03535348, 0.0127036 , ..., 0.06164382, 0.00763376,
        1.        ]])

In [19]:
from joblib import dump, load

def save_model(model, filename):
    joblib.dump(model, filename)

def load_model(filename):
    model = joblib.load(filename)
    return model


In [23]:
%%time
save_model(cosine_similarity_matrix, 'tf_idf.joblib')

CPU times: user 23.9 ms, sys: 245 ms, total: 269 ms
Wall time: 563 ms


In [20]:
%%time
new_data = joblib.load('../models/tf_idf.joblib')

CPU times: user 29.4 ms, sys: 142 ms, total: 171 ms
Wall time: 250 ms


In [24]:
new_data[7684]

array([0.01355815, 0.03535348, 0.0127036 , ..., 0.06164382, 0.00763376,
       1.        ])

In [33]:
for column in reversed(all_recipes.columns):
    print('Column Name : ', column, type(all_recipes[column][0]))
    # df[column] = df[column].apply(lambda x: np.array(x))
    

Column Name :  total_time <class 'numpy.int64'>
Column Name :  difficulty <class 'str'>
Column Name :  image <class 'str'>
Column Name :  category <class 'str'>
Column Name :  tags <class 'numpy.ndarray'>
Column Name :  author <class 'str'>
Column Name :  instructions <class 'list'>
Column Name :  ingredients <class 'bytes'>
Column Name :  cook_time <class 'numpy.int64'>
Column Name :  prep_time <class 'numpy.int64'>
Column Name :  diet <class 'str'>
Column Name :  course <class 'str'>
Column Name :  cuisine <class 'str'>
Column Name :  description <class 'str'>
Column Name :  rating <class 'numpy.float64'>
Column Name :  vote_count <class 'numpy.int64'>
Column Name :  record_health <class 'str'>
Column Name :  url <class 'str'>
Column Name :  recipe_title <class 'str'>


In [32]:
all_recipes['ingredients'] = all_recipes['ingredients'].apply(lambda x: x.tostring() if x is not None else x)


/var/folders/3j/_zy32sdx4s58w0nwxnq0zgvw0000gn/T/ipykernel_59549/73190061.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  all_recipes['ingredients'] = all_recipes['ingredients'].apply(lambda x: x.tostring() if x is not None else x)


In [34]:
all_recipes

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
0,Roasted Peppers And Mushroom Tortilla Pizza Recipe,https://www.archanaskitchen.com/roasted-peppers-and-mushroom-tortilla-pizza-recipe,good,434,4.791120,"Tortilla Pizza is a quicker version pizza to satisfy your cravings. It is a very quick and easy recipe for days that you do not feel like cooking a full fledged meal. With the preference of toppings of your choice this pizza recipe is definitely a winner at any home. The toppings used in this Roasted Peppers And Mushroom Tortilla Pizza has some roasted peppers, mushroom with loaded cheese and marinara sauce. Enjoy this easy recipe with your favorite toppings. This is a great recipe, if you are looking for an Indian/Fusion Pizza or a Homemade Pizza recipe. Serve Roasted Peppers And Mushroom Tortilla Pizza along with One Pot Tex-Mex Penne Pasta and Fennel Infused Mango Sorbet for a weekend night dinner.",Mexican,Dinner,Vegetarian,15,15,b'\xd0h{\x89\x02\x00\x00\x00 i{\x89\x02\x00\x00\x000\x80\xd0\x89\x02\x00\x00\x00pi{\x89\x02\x00\x00\x00\x90\x80\xd0\x89\x02\x00\x00\x000[n\x89\x02\x00\x00\x00\xc0i{\x89\x02\x00\x00\x00\xf0\x80\xd0\x89\x02\x00\x00\x00P\x81\xd0\x89\x02\x00\x00\x00',"[To begin making the Roasted Peppers And Mushroom Tortilla Pizza recipe, turn your oven setting to broiler and keep it ready., Grease your cast iron skillet with some olive oil and heat it over high heat., When it starts to shimmer bring down the heat to medium. Add olive oil and minced garlic to the marinara sauce and stir to combine, keep aside., Place one tortilla on the cast iron skillet. Spread a thin layer of marinara sauce over the tortilla., Spread the cheese as needed. When the cheese starts to melt, add in the toppings which includes onion, mushrooms, olives and bell peppers., Now shift the cast iron skillet to the oven which is already under broiler setting., When you see the cheese all melted and top starting to brown, remove and slice to serve., Serve Roasted Peppers And Mushroom Tortilla Pizza along with One Pot Tex-Mex Penne Pasta and Fennel Infused Mango Sorbet for a weekend night dinner.]",Divya Shivaraman,"[Party Food Recipes, Tea Party Recipes, Mushroom Recipes, Fusion Recipes, Tortilla Recipe, Bell Peppers Recipes]",Pizza Recipes,https://www.archanaskitchen.com/images/archanaskitchen/1-Author/divya_shivaraman/Roasted_peppers_and_Mushroom_Tortilla_Pizza.jpg,Easy,30
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy & Tangy Tomato Gravy,https://www.archanaskitchen.com/tomato-gotsu-recipe-spicy-tangy-tomato-gravy,good,3423,4.658654,"Tomato Gotsu Recipe also known as the Thakkali Curry is a quick and easy to make, spicy and tangy tomato gravy you can prepare for a change as a breakfast accompaniment or even as a side dish along with Ven Pongal, Khichdi, Paratha or Dosa. Tomato Gotsu that is packed with simple flavours from roasted baby onions simmered along with tamarind and sambar powder giving it that delectable taste. Serve Tomato Gotsu / Thakkali Curry Recipe with Ven Pongal and South Indian Filter Coffee for breakfast.",South Indian Recipes,Lunch,Vegetarian,10,20,b'\xb0\x81\xd0\x89\x02\x00\x00\x00\x10\x82\xd0\x89\x02\x00\x00\x00p\x82\xd0\x89\x02\x00\x00\x00\xd0\x82\xd0\x89\x02\x00\x00\x000\x83\xd0\x89\x02\x00\x00\x00pdd\x88\x02\x00\x00\x00\x90\x83\xd0\x89\x02\x00\x00\x00\x10j{\x89\x02\x00\x00\x00\xf0\x83\xd0\x89\x02\x00\x00\x000Pd\x88\x02\x00\x00\x00`j{\x89\x02\x00\x00\x00',"[To begin making Tomato Gotsu Recipe/ Thakkali Curry Recipe, prepare all the ingredients and keep them ready., Soak the tamarind in hot water for 20 minutes. Extract the water from the tamarind to get about 1/2 cup of thick tamarind., Roughly chop the tomatoes and pulse the chopped tomatoes in a mixer grinder. You want a chunky sauce and not a fine puree. This will help add texture to the Tomato Gotsu., Heat oil in a pressure cooker on medi